# US household debt indicators
> This notebook downloads, processes and charts data related to household debt from the US Federal Reserve. 

---

#### Import Python tools and Jupyter config

In [1]:
import os
import us
import json
import requests
import pandas as pd
# import jupyter_black
import altair as alt
import geopandas as gpd
import altair_cnn as altcnn
from IPython.display import Image
from datawrapper import Datawrapper

/Users/almatthews/.pyenv/versions/3.9.16/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/q0/1clc4jf97cxf003x4xgh89680000gr/T/ipykernel_57709/1261260140.py:8: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapel

In [2]:
# jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None
alt.themes.register("cnn", altcnn.theme)
alt.themes.enable("cnn")

ThemeRegistry.enable('cnn')

In [3]:
dw_token = os.environ.get("dw_api")
dw = Datawrapper(access_token=dw_token)
today = pd.Timestamp("today").strftime("%Y-%m-%d")

---

## Fetch indicators

#### 1. Quarterly household debt service ratio
> This [measure](https://fred.stlouisfed.org/series/TDSP) is the ratio of total required household debt payments to total disposable income.

In [4]:
debt_ratio_url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id=TDSP"

In [5]:
debt_ratio_df = pd.read_csv(debt_ratio_url, names=["date", "value"], header=0).round(2)

In [6]:
debt_ratio_df.head()

,date,value
0,1980-01-01,10.61
1,1980-04-01,10.63
2,1980-07-01,10.40
3,1980-10-01,10.25
4,1981-01-01,10.29


#### Last decade

In [7]:
recent = debt_ratio_df.query('date>"2014-12-31"').copy()

In [29]:
recent

,date,value
140,2015-01-01,10.00
141,2015-04-01,10.04
142,2015-07-01,10.10
143,2015-10-01,10.04
144,2016-01-01,10.00
145,2016-04-01,10.11
146,2016-07-01,10.16
147,2016-10-01,10.13
148,2017-01-01,10.08
149,2017-04-01,10.04


#### Export to Datawrapper

In [28]:
#### https://app.datawrapper.de/chart/tXaoi/publish

debt_ratio_id = "tXaoi"
dw.add_data(
    chart_id=f"{debt_ratio_id}",
    data=recent,
)

<Response [204]>

---

#### 2. Debt report from the [New York Fed Consumer Credit Panel/Equifax](https://www.newyorkfed.org/microeconomics/hhdc)

In [9]:
url = "https://www.newyorkfed.org/medialibrary/interactives/householdcredit/data/xls/HHD_C_Report_2023Q4"

In [10]:
xlsx_file = pd.read_excel(url, sheet_name="TABLE OF CONTENTS", skiprows=5, skipfooter=8)

In [11]:
sheet_names = pd.ExcelFile(url).sheet_names

---

## Delinquency rates

#### Percentage of debt 90+ days delinquent
> Aggregate delinquency rates increased in the fourth quarter of 2023. As of December, 3.1% of outstanding debt was in some stage of delinquency, up by 0.1 percentage point from the third quarter. Still, overall delinquency rates remain 1.6 percentage points lower than the fourth quarter of 2019.

In [12]:
del_df = (
    pd.read_excel(url, sheet_name="Page 12 Data", skiprows=3)
    .drop(["Unnamed: 8", "Unnamed: 9"], axis=1)
    .round(2)
).rename(
    columns={
        "Unnamed: 0": "quarter",
        "MORTGAGE": "mortgage",
        "HELOC": "heloc",
        "AUTO": "auto",
        "CC": "credit_card",
        "STUDENT LOAN": "student_loan",
        "OTHER": "other",
        "ALL": "all",
    }
)

#### Clean dates

In [13]:
del_df["year"] = "20" + del_df["quarter"].str.split(":", expand=True)[0]
del_df["quarter"] = del_df["quarter"].str.split(":", expand=True)[1]

In [14]:
del_melt = del_df.melt(
    id_vars=["year", "quarter"],
    value_vars=[
        "mortgage",
        "heloc",
        "auto",
        "credit_card",
        "student_loan",
        "other",
        "all",
    ],
)

In [15]:
del_melt["date"] = pd.to_datetime(
    del_melt["year"].astype(str) + del_melt["quarter"].str.replace("Q", "") + "1",
    format="%Y%m%d",
)

In [16]:
del_melt["value_pct"] = del_melt["value"] / 100
del_melt["variable"] = (
    del_melt["variable"]
    .str.replace("_", " ")
    .str.title()
    .str.replace("Heloc", "Home Equity")
)

In [17]:
del_melt.head()

,year,quarter,variable,value,date,value_pct
0,2003,Q1,Mortgage,1.21,2003-01-01,0.0121
1,2003,Q2,Mortgage,1.14,2003-02-01,0.0114
2,2003,Q3,Mortgage,1.10,2003-03-01,0.0110
3,2003,Q4,Mortgage,1.06,2003-04-01,0.0106
4,2004,Q1,Mortgage,1.01,2004-01-01,0.0101


In [18]:
alt.Chart(del_melt.query('variable != "All" and year > "2016"')).mark_area(
    interpolate="step"
).encode(
    x=alt.X(
        "date:T",
        title="",
        axis=alt.Axis(format="%Y", tickCount=2),
    ),
    y=alt.Y("value_pct:Q", title="", axis=alt.Axis(format="%", tickCount=3)),
    facet=alt.Facet("variable:N", columns=3, title=""),
).properties(
    width=150,
    height=150,
    title="Percentage in serious delinquency (90 days)",
)

alt.Chart(...)

---

## Total debt balance

#### By composition

In [19]:
debt_df = pd.read_excel(url, sheet_name="Page 3 Data", skiprows=3).rename(
    columns={"Unnamed: 0": "quarter"}
)
debt_df.columns = debt_df.columns.str.lower().str.replace(" ", "_")

In [20]:
debt_df["year"] = "20" + debt_df["quarter"].str.split(":", expand=True)[0]
debt_df["quarter"] = debt_df["quarter"].str.split(":", expand=True)[1]

In [21]:
debt_df["date"] = pd.to_datetime(
    debt_df["year"].astype(str) + debt_df["quarter"].str.replace("Q", "") + "1",
    format="%Y%m%d",
)

In [22]:
debt_melt = pd.melt(
    debt_df,
    id_vars=["year", "quarter", "date"],
    value_vars=[
        "mortgage",
        "he_revolving",
        "auto_loan",
        "credit_card",
        "student_loan",
        "other",
        # "total",
    ],
)

In [23]:
housing_variables = ["mortgage", "he_revolving"]

In [24]:
area = (
    alt.Chart(debt_melt.query(f"~variable.isin({housing_variables})"))
    .mark_area()
    .encode(x="date:T", y="value:Q", color="variable:N")
)

In [25]:
area

alt.Chart(...)

In [26]:
lines = (
    alt.Chart(debt_melt.query(f"~variable.isin({housing_variables})"))
    .mark_bar(interpolate="monotone")
    .encode(
        x=alt.X("date:T"), y=alt.Y("value:Q"), color=alt.Color("variable:N", title="")
    )
)
lines.configure_legend(orient="top")

alt.Chart(...)

In [46]:
debt_melt.query(f"~variable.isin({housing_variables})").pivot(index=['year','quarter','date'],columns='variable',values='value').reset_index().rename(columns={'auto_loan':'Auto','credit_card':'Credit card','student_loan':'Student loan','other':'Other'})

variable,year,quarter,date,Auto,Credit card,Other,Student loan
0,2003,Q1,2003-01-01,0.6410,0.6880,0.4776,0.240700
1,2003,Q2,2003-02-01,0.6220,0.6930,0.4860,0.242900
2,2003,Q3,2003-03-01,0.6840,0.6930,0.4773,0.248800
3,2003,Q4,2003-04-01,0.7040,0.6980,0.4486,0.252900
4,2004,Q1,2004-01-01,0.7200,0.6950,0.4465,0.259800
5,2004,Q2,2004-02-01,0.7430,0.6970,0.4231,0.262900
6,2004,Q3,2004-03-01,0.7510,0.7060,0.4100,0.330000
7,2004,Q4,2004-04-01,0.7280,0.7170,0.4229,0.345700
8,2005,Q1,2005-01-01,0.7250,0.7100,0.3941,0.363600
9,2005,Q2,2005-02-01,0.7740,0.7170,0.4024,0.374400


In [38]:
dw_debt=dw_debt

In [ ]:
#dw.add_data(chart_id='U2lZt',data=dw_debt)

In [41]:
dw_debt[dw_debt['date']>="2015-01-01"]

variable,year,quarter,date,Auto,Credit card,Other,Student loan
48,2015,Q1,2015-01-01,0.968,0.684,0.329,1.189
49,2015,Q2,2015-02-01,1.006,0.703,0.339,1.190
50,2015,Q3,2015-03-01,1.045,0.714,0.351,1.203
51,2015,Q4,2015-04-01,1.064,0.733,0.351,1.232
52,2016,Q1,2016-01-01,1.071,0.712,0.354,1.261
53,2016,Q2,2016-02-01,1.103,0.729,0.356,1.259
54,2016,Q3,2016-03-01,1.135,0.747,0.367,1.279
55,2016,Q4,2016-04-01,1.157,0.779,0.377,1.310
56,2017,Q1,2017-01-01,1.167,0.764,0.367,1.344
57,2017,Q2,2017-02-01,1.190,0.784,0.378,1.344


---

## Aggregate

#### Groupby state, etc.

---

## Charts

#### Save the chart

In [27]:
chart.save("visuals/chart.png")

NameError: name 'chart' is not defined

#### Make sure the chart is visible on Github

In [ ]:
Image(filename="visuals/chart.png")

---

## Metadata

#### Data provenance, column descriptions, etc.

---

## Exports

#### XyXy subset in CSV format to `processed`

#### JSON, GeoJSON, etc., to `processed`